# Quantum Kernel: Practical Guide

This notebook provides a didactic tour of MerLin's quantum kernel, using multiple datasets and construction methods.
We'll compute precomputed kernel matrices and train classical SVMs, and also show a short training loop with NKernelAlignment.

## Setup and helpers

In [1]:
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from merlin.algorithms.kernels import FidelityKernel, FeatureMap, KernelCircuitBuilder
from merlin.algorithms.loss import NKernelAlignment
from merlin.core.photonicbackend import PhotonicBackend
from merlin.core.generators import CircuitType

# Reproducibility
rng = np.random.RandomState(42)
torch.manual_seed(42)

def evaluate_kernel(kernel, X_train, X_test, y_train, y_test):
    """
        Compute precomputed kernel matrices and return accuracy.
    """
    X_train_t = torch.tensor(X_train, dtype=torch.float32)
    X_test_t = torch.tensor(X_test, dtype=torch.float32)
    K_train = kernel(X_train_t).detach().numpy()
    K_test = kernel(X_test_t, X_train_t).detach().numpy()
    clf = SVC(kernel="precomputed", random_state=42)
    clf.fit(K_train, y_train)
    return clf.score(K_test, y_test)

def check_kernel_properties(K):
    """
        Basic properties for sanity checking.
    """
    assert K.shape[0] == K.shape[1]
    assert np.allclose(K, K.T, atol=1e-6), "Kernel must be symmetric"
    # Diagonal close to 1, within tolerance
    assert np.allclose(np.diag(K), 1.0, atol=1e-1)
    return True


## Iris (multi-class) — quickstart

In [4]:
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

kernel = FidelityKernel.simple(
    input_size=4,      # number of features
    n_modes=6,         # photonic modes
    n_photons=2,       # photons (<= n_modes)
    circuit_type="series",
    reservoir_mode=False,
    force_psd=True,    # project to PSD if needed
)
# Evaluate
X_train_t = torch.tensor(X_train, dtype=torch.float32)
K_train = kernel(X_train_t).detach().numpy()
_ = check_kernel_properties(K_train)
acc = evaluate_kernel(kernel, X_train, X_test, y_train, y_test)
print(f'Iris accuracy: {acc:.3f}')


Iris accuracy: 1.000


## Iris — custom Perceval circuit (advanced)

In [5]:
import perceval as pcvl

def create_quantum_circuit(m, size=400):
    wl = pcvl.GenericInterferometer(
        m,
        lambda i: pcvl.BS() // pcvl.PS(pcvl.P(f'phase_1_{i}')) // pcvl.BS() // pcvl.PS(pcvl.P(f'phase_2_{i}')),
        shape=pcvl.InterferometerShape.RECTANGLE,
    )
    c = pcvl.Circuit(m)
    c.add(0, wl, merge=True)
    c_var = pcvl.Circuit(m)
    for i in range(size):
        px = pcvl.P(f'px-{i + 1}')
        c_var.add(i % m, pcvl.PS(px))
    c.add(0, c_var, merge=True)
    wr = pcvl.GenericInterferometer(
        m,
        lambda i: pcvl.BS() // pcvl.PS(pcvl.P(f'phase_3_{i}')) // pcvl.BS() // pcvl.PS(pcvl.P(f'phase_4_{i}')),
        shape=pcvl.InterferometerShape.RECTANGLE,
    )
    c.add(0, wr, merge=True)
    return c

def get_quantum_kernel(modes=10, input_size=4, photons=4, no_bunching=False):
    circuit = create_quantum_circuit(m=modes, size=input_size)
    feature_map = FeatureMap(
        circuit=circuit,
        input_size=input_size,
        input_parameters=['px'],
        trainable_parameters=['phase'],
        dtype=torch.float64,
    )
    input_state = [0] * modes
    for p in range(min(photons, modes // 2)):
        input_state[2 * p] = 1
    return FidelityKernel(
        feature_map=feature_map,
        input_state=input_state,
        no_bunching=no_bunching,
        force_psd=True,
    )

acc_custom = evaluate_kernel(
    get_quantum_kernel(input_size=4, modes=10, photons=4),
    X_train, X_test, y_train, y_test
)
print(f'Iris (custom circuit) accuracy: {acc_custom:.3f}')
print('Note: deep circuits may require larger tolerances due to numerical chains.')


Iris (custom circuit) accuracy: 0.967
Note: deep circuits may require larger tolerances due to numerical chains.


## Iris (binary) — training with NKernelAlignment

In [6]:
# Binary subset: classes 0 vs 1
mask = y < 2
X_b, y_b = X[mask], y[mask]
y_b_signed = 2 * y_b - 1  # {-1, +1}
Xtr, Xte, ytr, yte = train_test_split(X_b, y_b_signed, test_size=0.3, random_state=42, stratify=y_b_signed)

Xtr_t = torch.tensor(Xtr, dtype=torch.float64)
Xte_t = torch.tensor(Xte, dtype=torch.float64)
ytr_t = torch.tensor(ytr, dtype=torch.float32)

kernel_t = FidelityKernel.simple(input_size=4, n_modes=6, n_photons=2)
opt = torch.optim.Adam(kernel_t.parameters(), lr=1e-2)
loss_fn = NKernelAlignment()
for _ in range(3):
    opt.zero_grad()
    K = kernel_t(Xtr_t)
    loss = loss_fn(K, ytr_t)
    loss.backward()
    opt.step()

Ktr = kernel_t(Xtr_t).detach().numpy()
Kte = kernel_t(Xte_t, Xtr_t).detach().numpy()
clf = SVC(kernel="precomputed", random_state=42)
clf.fit(Ktr, ((ytr + 1) // 2))
acc_bin = clf.score(Kte, ((yte + 1) // 2))
print(f'Iris binary accuracy (after short training): {acc_bin:.3f}')


Iris binary accuracy (after short training): 1.000


## Wine (multi-class) — backend and builder

In [7]:
from sklearn.datasets import load_wine

wine = load_wine()
Xw, yw = wine.data, wine.target  # 13 features
Xw_tr, Xw_te, yw_tr, yw_te = train_test_split(Xw, yw, test_size=0.25, random_state=42, stratify=yw)

# Method 1: From PhotonicBackend
backend = PhotonicBackend(circuit_type=CircuitType.SERIES, n_modes=8, n_photons=3, reservoir_mode=True)
kernel_backend = FidelityKernel.from_photonic_backend(input_size=13, photonic_backend=backend)
acc_backend = evaluate_kernel(kernel_backend, Xw_tr, Xw_te, yw_tr, yw_te)
print(f'Wine (backend) accuracy: {acc_backend:.3f}')

# Method 2: Builder pattern
builder = KernelCircuitBuilder()
kernel_builder = (builder
    .input_size(13)
    .n_modes(8)
    .n_photons(3)
    .circuit_type(CircuitType.SERIES)
    .reservoir_mode(True)
    .build_fidelity_kernel()
)
acc_builder = evaluate_kernel(kernel_builder, Xw_tr, Xw_te, yw_tr, yw_te)
print(f'Wine (builder) accuracy: {acc_builder:.3f}')


Wine (backend) accuracy: 0.867
Wine (builder) accuracy: 0.867


## Breast Cancer (binary) — small sample demo

In [9]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
Xc, yc = cancer.data, cancer.target
Xc_tr, Xc_te, yc_tr, yc_te = train_test_split(Xc, yc, test_size=0.2, random_state=42, stratify=yc)

# Keep a small subset to keep computations light in docs builds
Xc_tr_s, yc_tr_s = Xc_tr[:40], yc_tr[:40]
Xc_te_s, yc_te_s = Xc_te[:20], yc_te[:20]

kernel_cancer = FidelityKernel.simple(input_size=Xc.shape[1], n_modes=12, n_photons=6, reservoir_mode=True)
acc_cancer = evaluate_kernel(kernel_cancer, Xc_tr_s, Xc_te_s, yc_tr_s, yc_te_s)
print(f'Breast Cancer (subset) accuracy: {acc_cancer:.3f}')


Breast Cancer (subset) accuracy: 0.550


## Tips: PSD projection and numerical stability

- Set `force_psd=True` to project symmetric kernels to positive semi-definite.
- Deep custom circuits can accumulate numerical error; relax tolerances when checking properties like diagonal ≈ 1.
- Use `reservoir_mode=True` for non-trainable, stable kernels; disable for trainable kernels with `NKernelAlignment`.